# Simple Backup Script for NX-OS
Requirements:
 - Must have SSH key generated on switch, and public key copied to backup server
 - Enable NXAPI and NXAPI-Sandbox on N7k
 
Program Flow: 
 - Script runs as a cron job once daily on backup server.
 - Script reads from list of tuples with device name, IP, username, and password.
 - Script connects to each switch using unencrypted HTTP basic auth (port 80).
 - Script issues command: copy startup-config sftp://10.18.188.188/home/bkup/n7k/[name]/[name]-[date]
 - N7k copies the startup-config to the backup server over ssh (sftp)
 - Script returns a simple HTTP status code if it cannot connect to NX-API.
 
To Do: 
 - Script verifies file
 - Script checks for device directory and creates if needed before copying.

On Switch: 
 - conf t
 - feature nxapi
 - feature nxapi-sandbox
 - username admin keypair generate rsa
 - show username admin keypair
 export keypair
 copy public key to bkup server
 add to authorized_keys
 test copy manually 

In [2]:
import requests
import json
import sys
import time
from datetime import date
from requests.auth import HTTPBasicAuth

## Variables

In [3]:
# Use time.time() for testing or when >1 day granularity is required.
# If backing up once per day or less, use date.today().
today = str(time.time())
#today = str(date.today())
print(today)

1562616975.779048


In [5]:
show_config_payload = {
  "ins_api": {
    "version": "1.2",
    "type": "cli_show",
    "chunk": "0",
    "sid": "1",
    "input": "show running-config",
    "output_format": "json"
  }
}

In [16]:
# This should be moved to a config file that can be read/parsed.
# Each tuple is a device hostname, ip address, and credentials.

devices = [
    ('N7K1-Labcore01','10.18.188.137','admin','password123'),
    ('N7K2-Labcore02','10.18.188.138','admin','correct horse battery staple'),
    ('N7K1-ACILab01','10.18.188.141','admin','password123'),
    ('N7K1-ACILab02','10.18.188.142','admin','correct horse battery staple'),       
    ('IPN1','10.18.188.143','admin','password123'),
    ('IPN2','10.18.188.144','admin','correct horse battery staple'),        
    ('MSITE1','10.18.188.145','admin','password123'),         
    ('MSITE2','10.18.188.146','admin','correct horse battery staple')
]

In [7]:
# Variables to construct the copy command. 
# Fill iin with your backup server ip/username, the proto (scp/SFTP), and the default_vrf.
# If no default VRF is needed, you may comment that line out.
bkup_host = "10.18.188.188"
bkup_user = "bkup"
proto_schema = "sftp"
bkup_source_file = "startup-config"
default_vrf = "vrf management"

In [20]:
# The main bit. 
# It loops over each device above, extracts the info and constructs the copy command 
# for each device, and finally creates the full JSON payload for the NX-API and sends it
# along with Basic HTTP Auth headers (*sigh*).

for device in devices:
        hostname = device[0]
        ip = device[1]
        username = device[2]
        password = device[3]
        #print(hostname + " " + ip + " " + username + " " + password)
        bkup_path = "/home/bkup/n7k/" + hostname + "/"
        bkup_dest_file = bkup_source_file + "_" + hostname + "_" + today + "-TESTING "
        input_cmd = "copy " + bkup_source_file + " " + proto_schema + "://" + bkup_user + "@" + bkup_host + bkup_path + bkup_dest_file + default_vrf
        #print(input_cmd)
        bkup_payload = {
            "ins_api": {
            "version": "1.2",
            "type": "cli_conf",
            "chunk": "0",
            "sid": "1",
            "input": input_cmd,
            "output_format": "json"
            }
        }
        

        auth = HTTPBasicAuth(username, password)
        
        nx_url = "http://" + ip + "/ins"
        print("Connecting to " + hostname + ".")
        print("Copying startup-config to " + bkup_dest_file)
        r = requests.post(nx_url, auth=auth, json=bkup_payload)
        
        # We can test for failure here and do something more if desired.
        if r.status_code == 200:
            print("Success!\n\n")
        else:
            print("Something's fucky.")
            print("HTTP Status: " + str(r.status_code))
            print("Womp womp.")

Connecting to N7K1-Labcore01.
Copying startup-config to startup-config_N7K1-Labcore01_1562616975.779048-TESTING 
Success!


Connecting to N7K2-Labcore02.
Copying startup-config to startup-config_N7K2-Labcore02_1562616975.779048-TESTING 
Success!


Connecting to N7K1-ACILab01.
Copying startup-config to startup-config_N7K1-ACILab01_1562616975.779048-TESTING 
Success!


Connecting to N7K1-ACILab02.
Copying startup-config to startup-config_N7K1-ACILab02_1562616975.779048-TESTING 
Success!


Connecting to IPN1.
Copying startup-config to startup-config_IPN1_1562616975.779048-TESTING 
Success!


Connecting to IPN2.
Copying startup-config to startup-config_IPN2_1562616975.779048-TESTING 
Success!


Connecting to MSITE1.
Copying startup-config to startup-config_MSITE1_1562616975.779048-TESTING 
Success!


Connecting to MSITE2.
Copying startup-config to startup-config_MSITE2_1562616975.779048-TESTING 
Something's fucky.
HTTP Status: 401
Womp womp.


In [15]:
# Jupyter notebook provides a stack trace if the sys.exit is triggered above.
#%tb

SyntaxError: unexpected character after line continuation character (<ipython-input-11-0db0a2461cbe>, line 1)

In [15]:
# Here's an example of the response headers provided by the N7k. 
# Note the Cookie is pretty simple compared to ACI.
#r.headers

{'Server': 'nginx/1.4.5', 'Date': 'Mon, 08 Jul 2019 17:40:09 GMT', 'Content-Type': 'text/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Set-Cookie': 'nxapi_auth=admin:156260820189115991'}

Success!


